In [89]:
from time import sleep
import serial

## Connects to the right serial port
Tries every serial port until it connects to the right one.

The following code blow is important for connecting the python script to the arduino.

In [94]:
ser = None
connected = False
for port in range(8):
    try:
        ser = serial.Serial('/dev/ttyACM%d' % port, 9600) # Establish the connection on a specific port
        connected = True
        print("Connected to device at /dev/ttyACM%d" % port)
        break
    except:
        continue
if not connected:
    print("Failed to connect")

Connected to device at /dev/ttyACM4


## Begin communication

Run the following code to initialize the serial connection key and ensure that it's connected.

In [95]:
KEY = b"$\r\n"

In [96]:
ser.write(KEY)
instr = b""
while instr != KEY:
    instr = ser.readline()
    print(instr)

b'\r\n'
b"Send a '$' to begin communication.\r\n"
b'Waiting for host....Found host.\r\n'
b'++++++++++++++++++++++++\r\n'
b'Allda Prototyping System\r\n'
b'(c) 2021 Allda\r\n'
b'Authors: Zachary Pitcher and Catherine Zeng\r\n'
b'$\r\n'


## Encoding Scheme

***What the arduino reads is a string "C100050C200050C300050D000"***

(written below with underscores so that it's easier to read)

C100050_C200050_C300050_D000

- The three "c1", "c2", "c3" describe the pressure from a scale of 0-15000 for each of the three channels
- The D figure describes the delay. ***This should not be lower than 15 or else the device may risk damage.***

The `MAX` and `MIN` variables denote the pressure range.

In [97]:
MAX = 15000
MIN = 0

## moveDildoPos
Moves the Dildo to a position

In [98]:
'''
This function takes a vector tuple argument and optional delay argument and moves the dildo to that location.
Example input:
moveDildoPos((MAX, 0, 0), 25)
'''
def moveDildoPos(vector, delay=20):
    ch1, ch2, ch3 = vector
    
    channels_string = "C1%05d" % ch1
    channels_string += "C2%05d" % ch2
    channels_string += "C3%05d" % ch3

    delay_string = "D%03d" % delay
    
    code = channels_string + delay_string
    code += "\r\n"
    ser.write(str.encode(code))
    
    instr = b""
    result = ""
       
    while instr != KEY:
        instr = ser.readline()
        result += instr.decode("utf-8")
    
    return result

In [127]:
def resetDildo():
    moveDildoPos((0, 0, 0), 30)

### Test cases for move DildoPos

In [138]:
# change your command here to see what happens
moveDildoPos((0, 0, MAX), 30)

'$\r\n'

In [129]:
resetDildo()

## moveDildoDegree
For a set number of degrees, this will move the dildo to that degree location.

In [101]:
POS_DICTIONARY = {
    "soft-center": (0, 0, 0),
    "hard-center": (MAX, MAX, MAX),
    0: (0, MAX, 0),
    30: (MAX, MAX, 0),
    60: (MAX, 0, 0),
    90: (MAX, 0, MAX),
    120: (0, 0, MAX),
    150: (0, MAX, MAX)
}

In [102]:
'''
pos is an angle that's a multiple of 30 degrees.
frequency is the number of taps that you want.
delay is the delay in microseconds 
'''
def moveDildoDegree(degree, delay=20):
    ch1, ch2, ch3 = POS_DICTIONARY[degree]
    
    channels_string = "C1%05d" % ch1
    channels_string += "C2%05d" % ch2
    channels_string += "C3%05d" % ch3

    delay_string = "D%03d" % delay
    
    code = channels_string + delay_string
    code += "\r\n"
    ser.write(str.encode(code))
    
    instr = b""
    result = ""
       
    while instr != KEY:
        instr = ser.readline()
        result += instr.decode("utf-8")
    
    return result

In [133]:
def swingDildo():
    for i in {0, 30, 60, 90, 120, 150, 0}:
        moveDildoDegree(i)
    resetDildo()

In [134]:
moveDildoDegree(30)

'$\r\n'

In [135]:
swingDildo()

## Pulse
This will pulse the dildo, getting it hard and then soft for a number of times.

In [136]:
def pulseDildo(pulseNumber):
    for i in range(pulseNumber):
        moveDildoPos((0, 0, 0))
        moveDildoPos((MAX, MAX, MAX))
    moveDildoPos((0, 0, 0))

In [137]:
pulseDildo(5)